# Python

This is an ipython notebook 

In [1]:
import timeit
import simplejson
from statistics import mean

In [2]:
fibonanchi__cache={}
def fibonanchi_cached(n):
    if n not in fibonanchi__cache:
        if n <= 1: fibonanchi__cache[n] = n
        else:      fibonanchi__cache[n] = fibonanchi_cached(n-1) + fibonanchi_cached(n-2) 
    return fibonanchi__cache[n]

def fibonanchi_uncached(n):
    if n <= 1: return n
    else:      return fibonanchi_uncached(n-1) + fibonanchi_uncached(n-2)
    
def fibonanchi_looped(n):
    fibonanchi__loop_cache = [0,1]
    if n >= len(fibonanchi__loop_cache): 
        for i in range(len(fibonanchi__loop_cache), n+1):
            fibonanchi__loop_cache.append( fibonanchi__loop_cache[i-1] + fibonanchi__loop_cache[i-2] )
    return fibonanchi__loop_cache[n]
    
def fibonanchi_range_cached(start=0,end=0):
    if start and not end: end = start; start = 0;
    return [ fibonanchi_cached(n) for n in range(start,end) ]
    
def fibonanchi_range_uncached(start=0,end=0):
    if start and not end: end = start; start = 0;
    return [ fibonanchi_uncached(n) for n in range(start,end) ]

def fibonanchi_range_looped(start=0,end=0):
    if start and not end: end = start; start = 0;
    return [ fibonanchi_looped(n) for n in range(start,end) ]

In [3]:
fibonanchi_looped(300)

222232244629420445529739893461909967206666939096499764990979600

In [4]:
fibonanchi_range_cached(30) == fibonanchi_range_uncached(30) == fibonanchi_range_looped(30)

True

In [5]:
fibonanchi_cached(30) == fibonanchi_uncached(30) == fibonanchi_looped(30)

True

#### Performance Tests

How high we can fibonanchi in a reasonable time?

In [6]:
funcs   = { "fibonanchi_uncached": fibonanchi_uncached, "fibonanchi_cached": fibonanchi_cached, "fibonanchi_looped": fibonanchi_looped }
timings = {}

# Note: fibonanchi_uncached(64) takes an inordinate amount of time (got bored waiting)
for name, func in funcs.items():
    try:
        time = 0
        n    = 1
        timings[name] = {}
        while time <= 0.5:
            time = timeit.timeit(lambda: func(2**n), number=1)
            timings[name][2**n] = time
            n    = n+1
            # print(name, 2**n, time)
    except Exception as exception: 
        # print(exception)
        pass
timings

{'fibonanchi_uncached': {2: 4.00003045797348e-06,
  4: 2.9999646358191967e-06,
  8: 8.000002708286047e-06,
  16: 0.0009370000334456563,
  32: 0.6484030000283383},
 'fibonanchi_cached': {2: 4.00003045797348e-06,
  4: 1.9999570213258266e-06,
  8: 2.00001522898674e-06,
  16: 2.00001522898674e-06,
  32: 4.00003045797348e-06,
  64: 4.600000102072954e-05,
  128: 3.6999990697950125e-05,
  256: 0.0001509999856352806,
  512: 0.00026599998818710446,
  1024: 0.00048800004879012704,
  2048: 0.0014329999685287476,
  4096: 0.0026950000319629908},
 'fibonanchi_looped': {2: 6.999995093792677e-06,
  4: 3.999972250312567e-06,
  8: 4.999979864805937e-06,
  16: 5.999987479299307e-06,
  32: 7.00005330145359e-06,
  64: 9.999959729611874e-06,
  128: 1.799996243789792e-05,
  256: 3.100000321865082e-05,
  512: 0.00011899997480213642,
  1024: 0.00020299997413530946,
  2048: 0.00036800000816583633,
  4096: 0.0007299999706447124,
  8192: 0.005231999966781586,
  16384: 0.02045200002612546,
  32768: 0.0864369999617

Now inspect relative timings for each doubling of the fibonanchi number

In [7]:
timings_diff = {}
for name, func in funcs.items():
    timings_diff[name] = {}
    for n in range(0,100):
        if 2**n in timings[name] and 2**(n+1) in timings[name]:
            timings_diff[name][2**(n+1)] = timings[name][2**(n+1)] / timings[name][2**n] 

timings_diff

{'fibonanchi_uncached': {4: 0.7499854481955762,
  8: 2.666699004637265,
  16: 117.12496452971864,
  32: 691.9989080938961},
 'fibonanchi_cached': {4: 0.4999854481955763,
  8: 1.0000291044558922,
  16: 1.0,
  32: 2.0,
  64: 11.499912689173458,
  128: 0.8043476060201905,
  256: 4.081081718856927,
  512: 1.761589493323465,
  1024: 1.834586731059806,
  2048: 2.9364750517576983,
  4096: 1.8806699868457994},
 'fibonanchi_looped': {4: 0.571425007691732,
  8: 1.2500036380040456,
  16: 1.2000023283158128,
  32: 1.166677984846574,
  64: 1.4285547979378015,
  128: 1.8000034924737192,
  256: 1.722225994948858,
  512: 3.838708466021751,
  1024: 1.705882496805915,
  2048: 1.8128081529731932,
  4096: 1.983695528386357,
  8192: 7.167123530376108,
  16384: 3.909021436539937,
  32768: 4.22633482550943,
  65536: 4.161250392760463,
  131072: 3.812836751504909}}

In [8]:
{ name: mean(timings_diff[name].values()) for name in timings_diff.keys() }

{'fibonanchi_uncached': 203.1351392691119,
 'fibonanchi_cached': 2.663516166335347,
 'fibonanchi_looped': 2.609784676568538}

fibonanchi_uncached() 
- grows exponentially due having to recompute the entire recursion tree. 
- It quickly becomes comutationally infeasable to calculate higher numbers.

fibonanchi_cached()
- should be O(N), meaning 2x time per doubling of N
- mean() is 2.1 (close to 2x) and the doubling time seems to go both up and down
- this variance may be related to hash map bucket resizing operations 

fibonanchi_looped()
- should be O(N), meaning 2x time per doubling of N
- mean() is 2.5 and the doubling time seems to increase with higher N
- this may be an artifiact of arrays needing a full memcopy on each resize event to preserve a contigious memory space/
- thus to grow an array from 0 to N, may require log(N) resizing events that each need to copy an average of 1/2 N memory
- this makes fibonanchi_looped() = O(N) + μO(N log(N))